_Data comes from **MIG/DualData**_

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pdb

from collections import namedtuple
from argparse import ArgumentParser
from path import Path
from IPython.display import display

In [2]:
plt.style.use('ggplot')
plt.figure()
pd.set_option('display.max_columns', 50)

# File name / Environment / Crowd / AI
Simset = namedtuple("Simset", ["Name","Envs","Crowd","AI"])
boolmap = lambda x:bool(int(x))
Situs = [
    Simset("All", *map(boolmap,"111")),
    Simset("Environment", *map(boolmap,"100")),
    Simset("Crowd", *map(boolmap,"010")),
    Simset("AI", *map(boolmap,"001")),
]

current = Situs[0] 

In [3]:
def parseLogfile(filename):
    benchmark = pd.read_table(filename, sep=' ', skiprows=1)
    benchmark = benchmark[benchmark["frames"] < benchmark["frames"].max()]
    
    value_col_names = ["hashing", "time_avg", "len_avg", "ple_avg", "cls_avg"]
    col_names = [col_name for col_name in benchmark.columns if 
        (col_name.endswith("th_obstacle") and current.Envs) or
        (col_name.endswith("th_region") and current.Crowd) or
        (col_name.endswith("th_ai") and current.AI)]
    
    # Modifing & extracting datas
    benchmark.loc[:, "hashing"] = benchmark[col_names].apply(
        lambda x: hash(tuple(x)),axis=1)
    benchmark.loc[:, "time_avg"] = (
        benchmark["agent_time_enableds"].apply(
        lambda x: pd.Series(x.strip("( )").split(','), dtype=float).mean()))
    benchmark.loc[:, "len_avg"] = (
        benchmark["agent_distance_traveleds"].apply(
        lambda x: pd.Series(x.strip("( )").split(','), dtype=float).mean()))
    benchmark.loc[:, "ple_avg"] = (
        benchmark["agent_ple_energys"].apply(
        lambda x: pd.Series(x.strip("( )").split(','), dtype=float).mean()))
    benchmark.loc[:, "cls_avg"] = (
        benchmark["collisionTimes"].apply(
        lambda x: pd.Series(x.strip("( )").split(','), dtype=float).mean()))

    sampleSet = benchmark[col_names + value_col_names]
    sample1, sample2 = sampleSet.iloc[::2], sampleSet.iloc[1::2]

    sample1.set_index('hashing', inplace=True)
    sample2.set_index('hashing', inplace=True)
    
    print("Dual trail")
    print("len of dataset", len((sample1-sample2).dropna())*2)
    print( (sample1-sample2).dropna().apply(np.std, ddof=1)
      .loc[value_col_names[1:]].to_frame() )
    
    print("Varience test")
    print(sampleSet.apply(np.std, ddof=1).loc[value_col_names[1:]].to_frame())
    
    print("Mean Value test")
    print(sampleSet.apply(np.mean).loc[value_col_names[1:]].to_frame())

In [5]:
for file in Path(".").glob("*.log"):
    print(file)
    parseLogfile(file)
    print("-"*20)

./MapB-ORCA-All-d.log
Dual trail
len of dataset 6722
                 0
time_avg  1.350545
len_avg   1.352537
ple_avg   5.755862
cls_avg   1.498189
Varience test
                  0
time_avg  22.934473
len_avg    7.974885
ple_avg   29.555360
cls_avg    3.644443
Mean Value test
                   0
time_avg  101.767456
len_avg   136.234687
ple_avg   474.022301
cls_avg    13.722719
--------------------
./MapB-SF-All-d.log
Dual trail
len of dataset 6752
                 0
time_avg  0.957641
len_avg   1.098175
ple_avg   3.839390
cls_avg   0.985135
Varience test
                  0
time_avg  11.543820
len_avg    7.695140
ple_avg   26.705820
cls_avg    3.529038
Mean Value test
                   0
time_avg  108.850866
len_avg   135.354259
ple_avg   459.898771
cls_avg     1.697489
--------------------
./MapA-ORCA-All-d.log
Dual trail
len of dataset 5840
                  0
time_avg   2.725200
len_avg    2.625872
ple_avg   10.079845
cls_avg    3.758721
Varience test
                  0
time_av